In [1]:
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
def to_correct_time(df):
    tot_rows = df.shape[0]
    for date_cell in range(0,tot_rows): 
        date_str = df.iloc[date_cell,0]
        date_format = '%d.%m.%Y %H:%M'
        date_obj = datetime.strptime(date_str, date_format)
        df.iloc[date_cell,0] = date_obj

In [24]:
mod = 'mod12'   # choose mod01, mod02, mod05, mod10__, mod12, mod17, mod18
pabs = {}
names = []

directory = "D:/ИТМО/Python/vosaline_votemper/data"   
for filename in os.listdir(directory):
    if 'csv' in filename:
        if mod in filename:
            path = f'{directory}/{filename}'
            name = path.split('/')[-1][:-4]
            names.append(name)
            data = pd.read_csv(path, sep=';', decimal='.', encoding='latin-1')
            df = pd.DataFrame(data)
            df = df.iloc[:, 1:]
            df.columns = df.columns.str[:11]
            to_correct_time(df)
            df = df.rename(columns={'vosaline-3.': 'vosaline-03', 'votemper-3.': 'votemper-03'})

            df['time'] = pd.to_datetime(df['time'], errors='coerce')
            only_winter = df.loc[(df['time'].dt.month.isin([1,2,12]))] #%Y-%m-%d %H:%M:%S
            only_winter = only_winter.drop('time', axis = 1)
            only_winter['col'] = np.arange(only_winter.shape[0])
            only_winter = only_winter.reset_index(drop=False)
            only_winter = only_winter.drop('index', axis = 1)
            pabs[name] = only_winter

#Merging with other tables 
df_merge = pd.DataFrame(columns=['col'])

for i in names:
    df_merge = pd.concat([df_merge, pabs[i]], axis = 1)
df_merge = df_merge.drop('col', axis = 1)
df_merge = df_merge.groupby(by=df_merge.columns, axis=1).mean()

# Mean for winter 
avg_winter = df_merge.mean(axis=0)

# DataFrame for winter values 
df_new = pd.DataFrame(avg_winter)
df_new['Index'] = df_new.index
df_new.reset_index(drop=True, inplace=True)
df_new.rename(columns={0: 'AvgWinter'}, inplace = True)
df_new["Param"] = df_new['Index'].str.split('-').str[0]
df_new["Depth"] = df_new['Index'].str.split('-').str[1].astype(float).round(2)
df_new_cols = ["Index","Param","Depth","AvgWinter"]
df_new = df_new[df_new_cols]
df_new.to_csv(f'D:/ИТМО/Python/vosaline_votemper/data/grouped/grouped_{mod}.csv')